# ACMA Scrapper
##### ©Haris Hassan

## Libraries

In [1]:
##=============================================================================
## ACMAscrapper V6.0.2
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#Author Haris Hassan
#Email haris.hassan@radhaz.com.au 
#linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================
# Import libraries

import re
import time
import requests
import random
import string
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
#Reset index of dataframe after sorting
import cProfile, pstats


## User Input

In [2]:
#### Replace Site 
sitecode = 35842

##### Filters
TRANSMITTER_ONLY = True

THIS_CLIENT_ONLY = False
THIS_CLIENT_ONLY_NAME = ''

IGNORE_CLIENT = False
IGNORE_CLIENT_NAME = ''

FREQUENCY_FILTER = False
MIN_FREQ = 650
MAX_FREQ = 3800

## Functions

In [3]:
def _init_acma_site(soup):
    SiteDetailsTitle, SiteDetails = scrape_table(soup, 1)
    SiteDetailsDictionary = {SiteDetailsTitle[i]: SiteDetails[i] for i in range(len(SiteDetailsTitle))}
    SiteDetailsDictionary['Location'] = ' '.join(SiteDetailsDictionary['Location'].split())
    return SiteDetailsDictionary
    
def get_all_assignments_at_this_Site(soup):
    ACMAtable = soup.find('table',{"class": "tablelist responsive"})
    ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
    ACMAdata = scrape_assignments_pages(pd.DataFrame(columns = ACMAheaders), soup)
    ACMAdata = ACMAdata.reset_index(drop=True)
    return ACMAdata

def scrape_page(url):
    """
    Scrape webpage from the given URL and return the parsed content.
    """
    try:
        response = session.get(url)
        if response.status_code == 200:
            return BeautifulSoup(response.text, 'lxml')
        else:
            print(f"Error: Failed to fetch data from {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {str(e)}")
        return None

def filter_assignments(ACMAdata, TRANSMITTER_ONLY, THIS_CLIENT_ONLY, 
                       THIS_CLIENT_ONLY_NAME, IGNORE_CLIENT, IGNORE_CLIENT_NAME,
                      FREQUENCY_FILTER, MAX_FREQ, MIN_FREQ):
    """
    Filter the antennas based on type, client and frequency
    """
    filteredAssignments = ACMAdata
    if TRANSMITTER_ONLY:
        filteredAssignments = filteredAssignments.loc[filteredAssignments['T/R']=='T']
    if THIS_CLIENT_ONLY:
        filteredAssignments = filteredAssignments.loc[filteredAssignments['Client']==THIS_CLIENT_ONLY_NAME]
    if IGNORE_CLIENT:
        filteredAssignments = filteredAssignments.loc[filteredAssignments['Client']!=IGNORE_CLIENT_NAME]
    if FREQUENCY_FILTER:
        filteredAssignments = filteredAssignments.loc[(filteredAssignments['Frequency (MHz)'] < MIN_FREQ) | (filteredAssignments['Frequency (MHz)']>MAX_FREQ)]
    return filteredAssignments

def freq_to_mhz(FREQ):
    """
    Convert all frequencies to MHz
    """
    frequency = []
    for x in FREQ:
        if 'GHz' in x:
            frequency.append(float(re.sub(' GHz','',x))*1000)
        elif 'MHz' in x:
            frequency.append(float(re.sub(' MHz','',x)))
    return frequency

def scrape_assignments_pages(ACMAdata, AssignmentsSoup):
    """
    Scrape all assignment entries in ACMA site assignment table
    """
    Assignments = AssignmentsSoup.find('table',{"class": "tablelist responsive"})
    AssignmentsHeaders = [td.text.strip() for td in Assignments.select('th')]
    ACMAdatap2 = pd.DataFrame(columns = AssignmentsHeaders)

    ACMAlinks = []
    for j in Assignments.find_all('tr')[1:]:
        AssignmentsValues = [tv.text.strip() for tv in j.find_all('td')]
        ACMAdatap2.loc[len(ACMAdatap2)] = AssignmentsValues
        ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
    ACMAdatap2.insert(1,'links', ACMAlinks)
    ACMAdata = pd.concat([ACMAdata, ACMAdatap2], axis=0)

    #Check if there's another Page
    NEXT_PAGE_LINK = ''.join(['https://web.acma.gov.au' + x for x in  [tl.get('href') for tl in AssignmentsSoup.findAll('a',{'title':"Next Page"})]])

    if NEXT_PAGE_LINK and NEXT_PAGE_LINK.strip():
        NEXT_PAGE_LINK = requests.get(NEXT_PAGE_LINK)
        print(NEXT_PAGE_LINK)
        return scrape_assignments_pages(ACMAdata, scrape_page(NEXT_PAGE_LINK))
    else:
        return ACMAdata
    
def start_Session():
    """
    Start the session reusing the underlying TCP connection for multiple requests as
    well as with gzip compression and randomly selected useragent 
    """
    session = requests.Session()
    session.headers.update({'Accept-Encoding': 'gzip'})
    session.headers.update({'User-Agent': UserAgents[random.randint(0,len(UserAgents)-1)]})
    return session

def scrape_table(webpage, n):
    """
    return the first column of the table as header list and 2nd column of table as values list
    """
    table_headers = [td.text for td in webpage.select_one('table:nth-of-type('+str(n)+')',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
    table_Values = [td.text.strip() for td in webpage.select_one('table:nth-of-type('+str(n)+')',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    return table_headers, table_Values

def find_link_destination(LinkedAssignmentsTable):
    """
    Checking if Antenna information page has destination table and return destination site name if it has
    otherwise return n/a
    """
    LinkToList = []    
    if LinkedAssignmentsTable:
        for j in LinkedAssignmentsTable.find_all('tr')[1:]:
            LinkToList.append([tv.text.strip() for tv in j.select('td:nth-of-type(5)')] )
            
    if not LinkToList:
        LinkToList = ['n/a']
        
    if LinkToList:
        LinkToListTemp = list(dict.fromkeys([''.join(p) for p in LinkToList]))
        for todel in LinkToListTemp:
            if SiteDetailsDictionary['Location'] in todel:
                LinkToListTemp.remove(todel)
        LinkToList = [str(LinkToListTemp)]
    return LinkToList

def add_antenna_to_table(AntennaTable, websoup, table, LinkedAssignmentsTable):
    """
    Scrape each antenna and add to antenna dataframe.
    """
    Antennaheaders, AntennaValues = scrape_table(websoup, table)
    Antennaheaders.extend(['Destination Link'])
    AntennaValues.extend(find_link_destination(LinkedAssignmentsTable))
    AntennaTable_toAdd = pd.DataFrame([{Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    return AntennaTable

def save_results(AntennaTable_export, filename):
    """
    Save the scraped and filtered results to a file with the given filename.
    """
    Renamed_headers = ['Device ID', 'Antenna', 'Client','Type','Freq (MHz)','Power','Height', 
                      'Polarisation','Azimuth', 'Tilt','Licence','Date Authorised','Destination Link']
    AntennaTable_export = AntennaTable_export.rename(columns={'Device Registration ID': 'Device ID', 'Device Type': 'Type', 
                                                          'Emission Center Frequency': 'Frequency'
                                                          ,'Transmitter Power': 'Power'
                                                          , 'Antenna Height (AGL)': 'Height', 
                                                          'Antenna Polarisation': 'Polarisation', 
                                                          'Antenna Azimuth': 'Azimuth',
                                                          'Licence Number': 'Licence','Antenna Tilt': 'Tilt'})
    AntennaTable_export = AntennaTable_export.reset_index(drop=True)
    AntennaTable_export.index += 1
    AntennaTable_export.to_excel(filename, columns = Renamed_headers, index=True)
    print(f"Results saved to {filename}")
    return AntennaTable_export[Renamed_headers]

In [4]:
UserAgents = [
    'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; SM-G996U Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; SM-G980F Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/78.0.3904.96 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 9; SM-G973U Build/PPR1.180610.011) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 8.0.0; SM-G960F Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.84 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 12; Pixel 6 Build/SD1A.210817.023; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/94.0.4606.71 Mobile Safari/537.36',
    'Mozilla/5.0 (Linux; Android 11; Pixel 5 Build/RQ3A.210805.001.A1; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/92.0.4515.159 Mobile Safari/537.36',
    'Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-T550 Build/LRX22G) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/3.3 Chrome/38.0.2125.102 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 7.0; SM-T827R4 Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.116 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 7.0; Pixel C Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 11; Lenovo YT-J706X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 12; SM-X906C Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',        
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

In [5]:
profiler = cProfile.Profile()
profiler.enable()

session = start_Session()
ACMAsite_url = 'https://web.acma.gov.au/rrl/site_search.site_lookup?pSITE_ID=' + str(sitecode)
soup = scrape_page(ACMAsite_url)
SiteDetailsDictionary = _init_acma_site(soup)
ACMAdata = get_all_assignments_at_this_Site(soup)

ACMAdata['Frequency (MHz)'] = freq_to_mhz(ACMAdata['Frequency'])

AcmaDataFiltered = filter_assignments(ACMAdata, TRANSMITTER_ONLY, THIS_CLIENT_ONLY, THIS_CLIENT_ONLY_NAME, IGNORE_CLIENT, IGNORE_CLIENT_NAME,FREQUENCY_FILTER, MAX_FREQ, MIN_FREQ)
ACMAdatalinks = list(dict.fromkeys(AcmaDataFiltered['links']) )
display(ACMAdatalinks)

['/rrl/assignment_search.lookup?pEFL_ID=5946752',
 '/rrl/assignment_search.lookup?pEFL_ID=10336375',
 '/rrl/assignment_search.lookup?pEFL_ID=10336379',
 '/rrl/assignment_search.lookup?pEFL_ID=10336383',
 '/rrl/assignment_search.lookup?pEFL_ID=10336387',
 '/rrl/assignment_search.lookup?pEFL_ID=10336391',
 '/rrl/assignment_search.lookup?pEFL_ID=10336395',
 '/rrl/assignment_search.lookup?pEFL_ID=10336363',
 '/rrl/assignment_search.lookup?pEFL_ID=10336367',
 '/rrl/assignment_search.lookup?pEFL_ID=10336371',
 '/rrl/assignment_search.lookup?pEFL_ID=5936554',
 '/rrl/assignment_search.lookup?pEFL_ID=6714703']

## Pulling Antennas Details from Web links 

In [6]:
AntennaTableHeader = ['Device Registration ID', 'Antenna', 'Client','Device Type','Emission Center Frequency',
                      'Transmitter Power', 'Antenna Height (AGL)','Antenna Polarisation','Antenna Azimuth', 
                      'Antenna Tilt','Licence Number','Date Authorised','Destination Link']

AntennaTable = pd.DataFrame(columns = AntennaTableHeader)
NotFoundLinks = []

for acmalink in ACMAdatalinks:
    st = time.time()
    try:
        soup2 = scrape_page('https://web.acma.gov.au' + acmalink)
        LinkedAssignmentsTable = soup2.find("table", {"class": "tablelist linked-responsive"})

        try:
            AntennaTable = add_antenna_to_table(AntennaTable, soup2, 1, LinkedAssignmentsTable)
        except Exception as e:
            print (e)
            print(acmalink)
            continue
        
        if acmalink[-2:]=='/1':
            try:
                AntennaTable = add_antenna_to_table(AntennaTable, soup2, 2, LinkedAssignmentsTable)
            except Exception as e:
                print(e)
                print(acmalink)
                continue
  
    except Exception as e:
        print(e)
        print('\nException')
        print(acmalink)
        print('\n')
        NotFoundLinks.append(acmalink)
        continue
        
    et = time.time()
    elapsed_time = et - st
    print(acmalink)
    print('Execution time:', elapsed_time, 'seconds')

profiler.disable()

/rrl/assignment_search.lookup?pEFL_ID=5946752
Execution time: 7.806344747543335 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336375
Execution time: 6.064845561981201 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336379
Execution time: 6.860693693161011 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336383
Execution time: 7.815019130706787 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336387
Execution time: 6.287982702255249 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336391
Execution time: 7.0478925704956055 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336395
Execution time: 6.683634996414185 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336363
Execution time: 7.6763596534729 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336367
Execution time: 7.040457725524902 seconds
/rrl/assignment_search.lookup?pEFL_ID=10336371
Execution time: 8.23487901687622 seconds
/rrl/assignment_search.lookup?pEFL_ID=5936554
Execution time: 6.428143739700317 seconds
/rrl/assignment_search.lo

In [7]:
AntennaTable

,Device Registration ID,Antenna,Client,Device Type,Emission Center Frequency,Transmitter Power,Antenna Height (AGL),Antenna Polarisation,Antenna Azimuth,Antenna Tilt,...,Antenna Multi Mode,Feeder Loss,EIRP,Mode,Callsign,Supplimental Device,Class of Station,Special Conditions,,Advisory Notes
0,NaN,"Parabolic-P, VHLP2-10W, ANDREW AUSTRALIA",NSW Police Force,Transmitter,10.6685 GHz,.0316 W pY,26 m,Linear Vertical,271.85°,,...,N,0 dB,75.9 W,Duplex,VKG,N,FX - Fixed station,NaN,NaN,NaN
1,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,418.6 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
2,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,418.85 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
3,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,419.1 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
4,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,419.35 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
5,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,419.6 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.\n\nThe level of power in the adjacent channel must not exceed -16dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
6,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,419.85 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm.",NOTE: There may be additional text attached to the licence\n associated with this assignment.,NaN
7,NaN,"Parallel array of vertical dipoles-D, IA404040-41-67, RF INDUSTRIES",NEW SOUTH WALES GOVERNMENT TELECOMMUNICATIONS AUTHORITY,Transmitter,414.85 MHz,80 W pY,30 m,Linear Vertical,,,...,N,5 dB,83 W,Duplex,NaN,N,FB - Base station,"The level of power in the adjacent channel must not exceed -16dBm.\n\nThe level of all discreet spurious components, measured at the output of the transmitter, must not exceed -30dBm

## Formatting the Data

In [8]:
AntennaTable = AntennaTable.fillna('')
AntennaTable['Antenna']=([''.join(x[2].title() +' '+ x[1] + ' '+ x[0].title()) for x in AntennaTable['Antenna'].str.split(',', 2)])

if 'EFL ID' in AntennaTable:
    AntennaTable['Device Registration ID'] = AntennaTable['Device Registration ID'].astype(str) + AntennaTable['EFL ID']
AntennaTable['Date Authorised']=([''.join(x[2] +'-'+ x[1] + '-'+ x[0].title()) for x in AntennaTable['Date Authorised'].str.split('/', 2)])

In [9]:
  
AntennaTable['Antenna']= AntennaTable['Antenna'].str.strip()
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Rf Industries', 'RFI')
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Australia', '')
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Parallel Array Of Vertical Dipoles', 'Dipole Array')
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('High Performance', '')
 
#AntennaTable['Client']=AntennaTable['Client'].apply(lambda x: x.title())
AntennaTable['Client']=AntennaTable['Client'].str.replace('Limited', 'Ltd')
AntennaTable['Client']=AntennaTable['Client'].str.replace('NEW SOUTH WALES GOVERNMENT', 'NSW')

AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jan', '01')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Feb', '02')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Mar', '03')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Apr', '04')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('May', '05')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jun', '06')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jul', '07')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Aug', '08')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Sep', '09')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Oct', '10')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Nov', '11')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Dec', '12')

AntennaTable['Antenna Polarisation']=AntennaTable['Antenna Polarisation'].str.replace('Linear', '')
AntennaTable['Transmitter Power']=AntennaTable['Transmitter Power'].str.replace(' pY', '')
AntennaTable['Freq (MHz)'] = freq_to_mhz(AntennaTable['Emission Center Frequency'])


## Export

In [15]:
filename = r'C:\Users\Mewtwo\Desktop'+'\\ACMA '+ str(sitecode) + " "+ re.sub("[\\\\/]", " ", SiteDetailsDictionary['Location']) + '.xlsx'
save_results(AntennaTable_export, filename)

In [19]:
display(NotFoundLinks)

[]

In [ ]:
stats = pstats.Stats(profiler).sort_stats('ncalls')
stats.print_stats()

In [ ]:
#AntennaTable_export.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', columns = Renamed_headers, index=True)
#import subprocess
#subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)